In [94]:
import numpy as np
import yfinance as yf
from scipy.stats import norm
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [95]:
def download_data(stock, start_date, end_date):
    data = pd.DataFrame()
    ticker = yf.download(stock, start_date, end_date)
    # ticker has lot of columns, reset_index and accept the adjusted close values
    data[stock] = ticker['Adj Close']
    return data

In [96]:
class VaRMonteCarlo:
    def __init__(self, Investment, mu, sigma, confidence_level, no_of_days, iterations):
        self.Investment = Investment
        self.mu = mu
        self.sigma = sigma
        self.confidence_level = confidence_level
        self.no_of_days = no_of_days
        self.iterations = iterations

    def simulation(self):
        rand = np.random.normal(0, 1, [1, self.iterations])
        
        # equation for the S(t) stock price
        # the random walk of our initial investment
        # captures the deterministic trend
        first_component = self.no_of_days * (self.mu - 0.5 * self.sigma**2)
        # represents stochastic movements
        second_component = self.sigma * np.sqrt(self.no_of_days) * rand

        asset_value = self.Investment * np.exp(first_component + second_component)

        # sort the stock price
        asset_value_sorted = np.sort(asset_value)

        # confidence level
        percentile = np.percentile(asset_value_sorted, (1-self.confidence_level)* 100)

        return self.Investment, percentile, asset_value

In [97]:
if __name__ == '__main__':

    # historical data to get mean and SD
    start_date = datetime.datetime(2014,1,1)
    end_date = datetime.datetime(2017,10,15)
    stock_data = download_data('C', start_date, end_date)
    stock_data['returns'] = stock_data['C'].pct_change().dropna()

    # we assume daily returns to be normally distributed
    mu = np.mean(stock_data['returns']) # drift/trend
    sigma = np.std(stock_data['returns'])

    Investment = 10000
    confidence_level = 0.99
    no_of_days = 1
    Iteration = 10000

    model = VaRMonteCarlo(Investment,mu,sigma,confidence_level,no_of_days,Iteration)
    # print(model.simulation())

    Investment, percentile, asset_value = model.simulation()
    print(Investment, percentile, Investment - percentile)

[*********************100%***********************]  1 of 1 completed

10000 9651.326354768318 348.6736452316818


In [98]:
# a = np.random.normal(0, 1, 10000)
# plt.hist(a, bins = 100)

# Visualization 

In [99]:
import hvplot.pandas
stock_data.hvplot.line(y='C', title='Historical Adjusted Close Prices', xlabel='Date', ylabel='Price')

:Curve   [Date]   (C)

In [100]:
stock_data['returns'].hvplot.hist(title='Distribution of Daily Returns', bins=50, xlabel='Returns', ylabel='Frequency')


:Histogram   [returns]   (Count)

In [101]:
var_data = pd.DataFrame({'Metric': ['Investment', 'VaR'], 'Value': [Investment, Investment - percentile]})
var_data.hvplot.bar(x='Metric', y='Value', title='Investment vs. VaR', ylabel='Value', color='Metric')


:Bars   [Metric]   (Value)

In [102]:
import panel as pn

pn.Column(
    stock_data.hvplot.line(y='C', title='Historical Adjusted Close Prices', xlabel='Date', ylabel='Price'),
    stock_data['returns'].hvplot.hist(title='Distribution of Daily Returns', bins=50, xlabel='Returns', ylabel='Frequency'),
    var_data.hvplot.bar(x='Metric', y='Value', title='Investment vs. VaR', ylabel='Value', color='Metric'),
).show()


Launching server at http://localhost:61505
